In [1]:
import psycopg2
import os

In [2]:
os.environ['POSTGRE_USERNAME'] = 'postgres'
os.environ['POSTGRE_PASSWORD'] = 'mahi'

In [3]:
USER = os.environ['POSTGRE_USERNAME']
PASSWORD = os.environ['POSTGRE_PASSWORD']

In [4]:
USER, PASSWORD

('postgres', 'mahi')

In [5]:
conn = psycopg2.connect(database='sql-data-analysis-and-visualisation-projects',
                                       user=USER,
                                       password=PASSWORD,
                                       host='localhost',
                                       port=5432)

In [6]:
conn

<connection object at 0x00000193D425CAE0; dsn: 'user=postgres password=xxx dbname=sql-data-analysis-and-visualisation-projects host=localhost port=5432', closed: 0>

In [8]:
all_ovens = getProductsByType(conn, 'oven')

In [9]:
len(all_ovens)

20

In [7]:
def getProductsByType(db_conn, prod_type):
    cursor = db_conn.cursor()
    sql_string = 'SELECT * FROM products WHERE product_type = %s'
    cursor.execute(sql_string, [prod_type])
    return (cursor.fetchall())

In [10]:
all_ovens[:5]

[(1, 'Reflector oven', 'oven'),
 (2, 'Convection microwave', 'oven'),
 (15, 'Masonry oven', 'oven'),
 (24, 'Microwave oven', 'oven'),
 (32, 'Russian oven', 'oven')]

In [11]:
# Avoiding N+1 queries
# Let's say we want to get all product orders which include any type of ovens

In [12]:
# global cursor
cursor = conn.cursor()
fetchall_count = 0

for oven in all_ovens:
    oven_id = oven[0]
    sql_string = 'SELECT * FROM product_orders WHERE product_id = %s'
    
    cursor.execute(sql_string, [oven_id])
    oven_orders = cursor.fetchall()
    fetchall_count += 1
    
    print('fetchall_count: ', fetchall_count)
    print(oven_orders, "\n")


fetchall_count:  1
[(3, 1, 5, Decimal('72.69')), (25, 1, 4, Decimal('63.59')), (59, 1, 3, Decimal('22.07')), (68, 1, 4, Decimal('196.11')), (71, 1, 5, Decimal('227.35')), (81, 1, 4, Decimal('116.89')), (121, 1, 2, Decimal('277.29')), (123, 1, 1, Decimal('127.55')), (138, 1, 1, Decimal('38.29')), (155, 1, 1, Decimal('279.34')), (187, 1, 3, Decimal('149.28')), (203, 1, 1, Decimal('299.98')), (208, 1, 1, Decimal('121.73')), (259, 1, 1, Decimal('167.60')), (300, 1, 1, Decimal('66.57'))] 

fetchall_count:  2
[(20, 2, 1, Decimal('211.07')), (57, 2, 4, Decimal('65.81')), (58, 2, 5, Decimal('102.97')), (63, 2, 4, Decimal('43.22')), (79, 2, 2, Decimal('293.17')), (80, 2, 1, Decimal('181.54')), (94, 2, 4, Decimal('297.49')), (130, 2, 3, Decimal('44.08')), (141, 2, 3, Decimal('187.80')), (155, 2, 2, Decimal('24.27')), (185, 2, 3, Decimal('246.71')), (237, 2, 3, Decimal('224.47')), (253, 2, 4, Decimal('298.50')), (263, 2, 2, Decimal('112.06')), (273, 2, 2, Decimal('182.99'))] 

fetchall_count:  3


In [13]:
del cursor

In [14]:
# NOTE:

# In above query, we are like check against one item and get all orders of specific item. Drop them back at home.

# Then again go back to store and check against another item and get all orders of that item. Drop them back at home again.

# This make inefficient and we want to get all orders of all related items while querying and bring them back to home all together.

In [15]:
# more efficient way
def getProductOrdersByType(db_conn, product_type):
    cursor = db_conn.cursor()
    
    sql_string = 'SELECT po.* \
                        FROM product_orders po \
                        JOIN products p \
                        ON p.product_id = po.product_id  \
                        WHERE p.product_type = %s'
    
    cursor.execute(sql_string, [product_type])
    return (cursor.fetchall())


In [16]:
all_product_orders = getProductOrdersByType(conn, 'oven')

In [17]:
len(all_product_orders)

249

In [18]:
all_product_orders[:5]

[(3, 1, 5, Decimal('72.69')),
 (25, 1, 4, Decimal('63.59')),
 (59, 1, 3, Decimal('22.07')),
 (68, 1, 4, Decimal('196.11')),
 (71, 1, 5, Decimal('227.35'))]

In [19]:
# Challenge
# write a function that executes an SQL query to reterive all product orders.
# type: fryer
# quanity greater than 3

In [20]:
def getAllProductOrdersByType(db_conn, product_type, quantity_greater_than):
    
    cursor = db_conn.cursor()
    sql_string = 'SELECT po.* \
                        FROM product_orders po \
                        JOIN products p \
                        ON p.product_id = po.product_id \
                        WHERE p.product_type = %s \
                        AND po.quantity > %s'
    
#     sql_string = 'SELECT po.* \
#                         FROM product_orders po \
#                         JOIN products p \
#                         ON p.product_id = po.product_id  \
#                         WHERE p.product_type = %s \
#                         AND po.quantity > 3'
    
    cursor.execute(sql_string, [product_type, quantity_greater_than])
    return (cursor.fetchall())

In [21]:
all_product_orders = getAllProductOrdersByType(conn, 'fryer', 3)

In [22]:

len(all_product_orders)

19

In [23]:
all_product_orders

[(10, 3, 4, Decimal('90.93')),
 (24, 3, 5, Decimal('131.20')),
 (86, 3, 5, Decimal('266.13')),
 (140, 3, 5, Decimal('298.42')),
 (150, 3, 5, Decimal('259.76')),
 (151, 3, 4, Decimal('262.93')),
 (273, 3, 4, Decimal('222.86')),
 (56, 17, 5, Decimal('64.47')),
 (220, 17, 5, Decimal('146.25')),
 (224, 17, 4, Decimal('30.91')),
 (295, 17, 5, Decimal('143.12')),
 (254, 26, 4, Decimal('22.24')),
 (75, 54, 4, Decimal('165.38')),
 (98, 54, 4, Decimal('45.03')),
 (150, 54, 5, Decimal('62.33')),
 (7, 64, 5, Decimal('261.31')),
 (90, 64, 5, Decimal('133.20')),
 (183, 64, 5, Decimal('58.37')),
 (205, 64, 5, Decimal('228.90'))]